<a href="https://colab.research.google.com/github/GabrielCalcada/astropytutorialsptbr/blob/main/tutorials/astropy-coordinates/1-Coordinates-Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coordenadas Astronômicas 1: Introdução com astropy.coordinates

## Autores
Adrian Price-Whelan

## Tradução para o Português
Gabriel Valim Calçada

## Metas de aprendizagem
* Criar objetos `astropy.coordinates.SkyCoord` usando dados de coordenadas e nomes de objetos
* Use objetos SkyCoord para se familiarizar com a programação orientada a objetos (object oriented programming (OOP))
* Use o objeto `SkyCoord` para consultar o arquivo *Gaia* usando `astroquery`
* Dados de coordenadas de saída em diferentes representações de string
* Demonstrar como trabalhar com coordenadas do céu 3D (incluindo informações de distância para objetos)

## Keywords - Palavras-chave
coordenadas, OOP, astroconsulta, gaia


## Resumo

Os astrônomos usam uma ampla variedade de sistemas de coordenadas e formatos para representar as coordenadas do céu de objetos celestes. Por exemplo, você pode ter visto termos como "right ascension" e "declination" ou "galactic latitude and longitude," e pode ter visto componentes de coordenadas angulares representados como "0h39m15.9s", "00:39:15.9" ou 9,81625º. O subpacote `astropy.coordinates` fornece ferramentas para representar as coordenadas de objetos e transformá-las entre diferentes sistemas.

Neste tutorial, vamos explorar como o pacote `astropy.coordinates` pode ser usado para trabalhar com coordenadas astronômicas. Você pode achar útil manter [a documentação do Astropy para o pacote de coordenadas](http://docs.astropy.org/en/stable/coordinates/index.html) aberta junto com este tutorial para referência ou leitura adicional. No texto abaixo, você também pode ver alguns links que se parecem com ([docs](http://docs.astropy.org/en/stable/coordinates/index.html)). Esses links levarão você a partes da documentação diretamente relevantes para as células das quais eles se vinculam.

*Obs: Este é o primeiro tutorial de uma série de tutoriais sobre astropy.coordinates.*
- [Próximo tutorial: Coordenadas Astronômicas 2: Transformando Sistemas e Representações de Coordenadas](2-Coordinates-Transforms)

## Imports

Começamos importando alguns pacotes que precisaremos abaixo:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file

from astroquery.gaia import Gaia
Gaia.ROW_LIMIT = 10000  # Set the row limit for returned data

## Representando posições no céu com `astropy.coordinates`

No Astropy, a maneira mais comum de representar e trabalhar com coordenadas do céu é usar o objeto `SkyCoord` ([docs](https://docs.astropy.org/en/stable/coordinates/skycoord.html)). Um `SkyCoord` pode ser criado diretamente a partir de ângulos ou matrizes de ângulos com unidades associadas, conforme demonstrado abaixo.

Para começar, vamos supor que queremos criar um objeto `SkyCoord` para o centro do aglomerado de estrelas aberto NGC 188 para que mais tarde possamos consultar e recuperar estrelas que possam ser membros do aglomerado. Vamos supor também, por enquanto, que já sabemos que as coordenadas no céu do aglomerado são (12,11, 85,26) graus no frame de coordenadas do ICRS. O ICRS — às vezes referido como coordenadas "equatoriais" ou "J2000" ([mais informações sobre o ICRS](https://arxiv.org/abs/astro-ph/0602086)) — é atualmente o frame de coordenadas astronômicas mais comum para astronomia estelar ou extragaláctica, e é o frame de coordenadas padrão para `SkyCoord`. Como já sabemos a posição do ICRS do NGC 188 (veja acima), podemos criar um objeto `SkyCoord` para o cluster passando os dados para o inicializador `SkyCoord`:

In [ ]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg)
ngc188_center

Mesmo que o frame padrão seja ICRS, geralmente é recomendado especificar explicitamente o quadro em que suas coordenadas estão. Neste caso, esta seria uma maneira equivalente de criar nosso objeto `SkyCoord` para NGC 188:

In [ ]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg, frame='icrs')
ngc188_center

Como veremos mais adiante nesta série, existem muitos outros frame de coordenadas suportados, por isso ajuda a adquirir o hábito de passar o nome de um frame de coordenadas.

Nas inicializações acima, passamos objetos `astropy.units.Quantity` com unidades angulares para especificar os componentes angulares de nossas coordenadas do céu. O inicializador `SkyCoord` também aceitará coordenadas formatadas em strings como strings separadas para Right Ascension  (RA) e Declination  (Dec) ou uma única string. Por exemplo, se tivermos dados de coordenadas do céu sexagesimais: neste caso, a representação dos dados inclui especificações das unidades (o "hms" para "hora minuto segundo" e o "dms" para "graus minuto segundo" (degrees minute second)

In [ ]:
SkyCoord('00h48m26.4s', '85d15m36s', frame='icrs')

Algumas representações de string não definem explicitamente as unidades, então às vezes é necessário especificar as unidades dos dados de coordenadas da string explicitamente se elas não estiverem incluídas implicitamente:

In [ ]:
SkyCoord('00:48:26.4 85:15:36', unit=(u.hour, u.deg), 
         frame='icrs')

Para obter mais informações e exemplos sobre como inicializar objetos `SkyCoord`, [consulte esta documentação](http://docs.astropy.org/en/latest/coordinates/skycoord.html).

Para as inicializações do `SkyCoord` demonstradas acima, assumimos que já tínhamos os valores dos componentes de coordenadas prontos. Se você não conhece os valores das coordenadas e o objeto que você está interessado está em [SESAME](http://cdsweb.u-strasbg.fr/cgi-bin/Sesame), você também pode procurar e carregar valores de coordenadas automaticamente do nome do objeto usando o método de classe `SkyCoord.from_name()`<sup>1</sup> ([docs](http://docs.astropy.org/en/latest/coordinates/index.html# métodos de conveniência)). Observe, no entanto, que isso requer uma conexão com a Internet. É seguro pular esta célula se você não estiver conectado à internet porque já definimos o objeto `ngc188_center` nas células acima.

<sub> <sup>1</sup>Se você não sabe o que é um método de classe, pense nele como um construtor alternativo para um objeto `SkyCoord` — chamar `SkyCoord.from_name()` com um nome lhe dá um novo objeto `SkyCoord`. Para obter informações mais detalhadas sobre o que são métodos de classe e quando são úteis, consulte [esta página](https://julien.danjou.info/blog/2013/guide-python-static-class-abstract-methods).</sub>

In [ ]:
ngc188_center = SkyCoord.from_name('NGC 188')
ngc188_center

O objeto `SkyCoord` que definimos agora tem várias maneiras de acessar as informações de coordenadas contidas nele. Todos os objetos `SkyCoord` possuem atributos que permitem que você recupere os dados do componente de coordenadas, mas os nomes dos componentes mudarão dependendo do quadro de coordenadas do `SkyCoord` que você possui. Em nossos exemplos, criamos um `SkyCoord` no quadro ICRS, então os nomes dos componentes são abreviações em minúsculas de Right Ascension, `.ra`, e Declination, `.dec`:

In [ ]:
ngc188_center.ra, ngc188_center.dec

Os atributos do componente `SkyCoord` (aqui ``ra`` e ``dec``) retornam objetos especializados do tipo `Quantity` que facilitam o trabalho com dados angulares. Enquanto `Quantity` ([docs](http://docs.astropy.org/en/stable/units/index.html)) é uma classe geral que representa valores numéricos e unidades físicas de qualquer tipo, `astropy.coordinates` define subclasses de `Quantity` que são projetadas especificamente para trabalhar com ângulos, como o `Angle` ([docs](http://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html) ) classe. A classe `Angle` tem subclasses adicionais e mais especializadas `Latitude` ([docs](http://docs.astropy.org/en/stable/api/astropy.coordinates.Latitude.html)) e `Longitude` ( [docs](http://docs.astropy.org/en/stable/api/astropy.coordinates.Longitude.html)). Esses objetos armazenam ângulos, fornecem atributos úteis para converter rapidamente em unidades angulares comuns e permitem a formatação dos valores numéricos em vários formatos. Por exemplo, em um notebook Jupyter, esses objetos sabem como se representar usando LaTeX:

In [ ]:
ngc188_center.ra

In [ ]:
ngc188_center.dec

In [ ]:
type(ngc188_center.ra), type(ngc188_center.dec)

Com esses objetos, podemos recuperar os componentes de coordenadas em diferentes unidades usando o método `Quantity.to()`:

In [ ]:
(ngc188_center.ra.to(u.hourangle), 
 ngc188_center.ra.to(u.radian),
 ngc188_center.ra.to(u.degree))

Ou usando os atributos abreviados, que retornam apenas os valores dos componentes:

In [ ]:
(ngc188_center.ra.hour, 
 ngc188_center.ra.radian, 
 ngc188_center.ra.degree)

Também podemos formatar os valores em strings com unidades especificadas ([docs](http://docs.astropy.org/en/latest/coordinates/formatting.html)), por exemplo:

In [ ]:
ngc188_center.ra.to_string(unit=u.hourangle, sep=':', pad=True)

## Consultando o arquivo *Gaia* para recuperar as coordenadas das estrelas no NGC 188

Agora que temos um objeto `SkyCoord` para o centro de NGC 188, podemos usar este objeto com o pacote `astroquery` para consultar diversos bancos de dados astronômicos (veja uma lista completa de [serviços disponíveis na documentação do astroquery](https: //astroquery.readthedocs.io/en/latest/#available-services)). Aqui, usaremos o objeto `SkyCoord` `ngc188_center` para selecionar fontes do catálogo *Gaia* Data Release 2 em torno da posição do centro de NGC 188 para procurar estrelas que possam ser membros do aglomerado estelar. Para isso, usaremos o subpacote `astroquery.gaia` ([docs](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html)).

Isso requer uma conexão com a Internet, mas se falhar, o arquivo de catálogo é incluído no repositório para que você possa carregá-lo localmente (pule a próxima célula se não tiver uma conexão com a Internet):

In [ ]:
job = Gaia.cone_search_async(ngc188_center, radius=0.5*u.deg)
ngc188_table = job.get_results()

# only keep stars brighter than G=19 magnitude
ngc188_table = ngc188_table[ngc188_table['phot_g_mean_mag'] < 19*u.mag]

In [ ]:
cols = [
    'source_id',
    'ra',
    'dec',
    'parallax',
    'parallax_error',
    'pmra',
    'pmdec',
    'radial_velocity',
    'phot_g_mean_mag',
    'phot_bp_mean_mag',
    'phot_rp_mean_mag'
]
ngc188_table[cols].write('gaia_results.fits', overwrite=True)

A célula acima pode não funcionar se você não tiver conexão com a internet, por isso incluímos a tabela de resultados junto com o notebook:

In [ ]:
ngc188_table = QTable.read('gaia_results.fits')

In [ ]:
len(ngc188_table)

O objeto `astropy.table` `Table` retornado agora contém cerca de 5000 estrelas de *Gaia* DR2 em torno da posição coordenada do centro de NGC 188. Vamos agora construir um objeto `SkyCoord` com a tabela de resultados. No arquivo de dados *Gaia*, as coordenadas ICRS de uma fonte são fornecidas como nomes de coluna `"ra"` e `"dec"`:

In [ ]:
ngc188_table['ra']

In [ ]:
ngc188_table['dec']

Observe que, como o arquivo *Gaia* fornece tabelas de dados com unidades associadas, e lemos essa tabela usando o objeto `QTable` ([docs](http://docs.astropy.org/en/latest/table/mixin_columns. html#quantity-and-qtable)), as colunas da tabela acima são representadas como objetos `Quantity` com unidades de graus. Observe também que essas colunas contêm muitos valores de coordenadas (>5000!). Podemos passá-los diretamente para `SkyCoord` para obter um único objeto `SkyCoord` para representar todas essas coordenadas:

In [ ]:
ngc188_gaia_coords = SkyCoord(ngc188_table['ra'], ngc188_table['dec'])
ngc188_gaia_coords

### Exercícios

Crie um 'SkyCoord' para o centro do aglomerado aberto das Plêiades (olhando as coordenadas e passando-as, ou usando o método de conveniência que aprendemos acima):

In [ ]:
ngc188_center = SkyCoord.from_name('NGC 188')

Usando apenas uma única chamada de método/função no objeto `SkyCoord` representando o centro de NGC 188, imprima uma string com o RA/Dec no formato 'HH:MM:SS.S DD:MM:SS.S'. Verifique sua resposta no [SIMBAD](http://simbad.u-strasbg.fr/simbad/), que mostrará as coordenadas sexagesimais do objeto.

<sub>(Dica: `SkyCoord.to_string()` pode ser útil)</sub>

In [ ]:
ngc188_center.to_string(style="hmsdms", sep=":", precision=1)

Usando uma única chamada de método/função no objeto `SkyCoord` contendo os resultados de nossa consulta *Gaia*, calcule a separação angular entre cada estrela resultante e as coordenadas do centro do cluster para NGC 188.

<sub>(Dica: `SkyCoord.separation()` pode ser útil)</sub>

In [ ]:
ngc188_gaia_coords.separation(ngc188_center)

## Mais do que apenas posições no céu: incluindo informações de distância no `SkyCoord` `Código in-line`

Até agora, usamos `SkyCoord` para representar posições angulares do céu (ou seja, `ra` e `dec` apenas). Às vezes é útil incluir informações de distância com as coordenadas do céu de uma fonte, especificando assim a posição 3D completa de um objeto. Para passar informações de distância, `SkyCoord` aceita o argumento de palavra-chave "`distance`". Então, se soubéssemos que a distância para NGC 188 é 1,96 kpc, também poderíamos passar uma distância (como um objeto `Quantity`) usando este argumento:

In [ ]:
ngc188_center_3d = SkyCoord(12.11*u.deg, 85.26*u.deg,
                            distance=1.96*u.kpc)

Com a tabela de dados de *Gaia* que recuperamos acima para estrelas ao redor de NGC 188, `ngc188_table`, também temos medições de paralaxe para cada estrela. Para uma paralaxe $\varpi$ medida com precisão, a distância $d$ até uma estrela pode ser obtida aproximadamente como $d \approx 1/\varpi$. Isso só funciona se o erro de paralaxe for pequeno em relação à paralaxe ([veja a discussão neste artigo](https://arxiv.org/abs/1507.02105)), então, se quisermos usar essas paralaxes para obter distâncias, primeiro tem que filtrar as estrelas que têm baixa paralaxe sinal-ruído:

In [ ]:
parallax_snr = ngc188_table['parallax'] / ngc188_table['parallax_error']
ngc188_table_3d = ngc188_table[parallax_snr > 10] 
len(ngc188_table_3d)

A seleção acima em `parallax_snr` mantém estrelas que têm uma medida de paralaxe de ~10 sigma, mas este é um limite de seleção arbitrário que você pode querer ajustar ou remover em seus próprios casos de uso. Essa seleção removeu mais da metade das estrelas em nossa tabela original, mas para as estrelas restantes podemos ter certeza de que converter as medidas de paralaxe em distâncias é mais seguro.

A maneira padrão de passar uma distância para um objeto `SkyCoord`, como acima, é passar uma `Quantidade` com uma unidade de comprimento. No entanto, `astropy.coordinates` também fornece um objeto especializado, `Distance`, para lidar com transformações comuns de diferentes representações de distância ([docs](http://docs.astropy.org/en/latest/coordinates/index.html# distância)). Entre outras coisas, esta classe suporta passar um valor de paralaxe:

In [ ]:
Distance(parallax=1*u.mas)

O catálogo de estrelas que consultamos em *Gaia* contém informações de paralaxe em unidades de milissegundos, para que possamos criar um objeto `Distance` diretamente desses valores:

In [ ]:
gaia_dist = Distance(parallax=ngc188_table_3d['parallax'].filled(np.nan))

Podemos então criar um objeto `SkyCoord` para representar as posições 3D de todas as estrelas *Gaia* passando este objeto de distância para o inicializador `SkyCoord`:

In [ ]:
ngc188_coords_3d = SkyCoord(ra=ngc188_table_3d['ra'], 
                            dec=ngc188_table_3d['dec'],
                            distance=gaia_dist)
ngc188_coords_3d

Vamos agora usar `matplotlib` para traçar as posições do céu de todas essas fontes, coloridas pela distância para enfatizar as estrelas do aglomerado:

In [ ]:
fig, ax = plt.subplots(figsize=(6.5, 5.2),
                       constrained_layout=True)
cs = ax.scatter(ngc188_coords_3d.ra.degree, 
                ngc188_coords_3d.dec.degree, 
                c=ngc188_coords_3d.distance.kpc, 
                s=5, vmin=1.5, vmax=2.5, cmap='twilight')
cb = fig.colorbar(cs)
cb.set_label(f'distance [{u.kpc:latex_inline}]')

ax.set_xlabel('RA [deg]')
ax.set_ylabel('Dec [deg]')

ax.set_title('Gaia DR2 sources near NGC 188', fontsize=18)

Agora que temos informações de posição 3D para o centro do cluster e para as estrelas que consultamos em *Gaia*, podemos calcular a separação 3D (distância) entre todas as fontes de *Gaia* e o centro do cluster:

In [ ]:
sep3d = ngc188_coords_3d.separation_3d(ngc188_center_3d)
sep3d

### Exercícios

Usando os valores de separação 3D, defina uma máscara booleana para selecionar os membros candidatos do cluster. Selecione todas as estrelas dentro de 50 pc do centro do cluster. Quantos membros candidatos da NGC 188 temos, com base em suas posições 3D?

In [ ]:
ngc188_3d_mask = sep3d < 50*u.pc
ngc188_3d_mask.sum()

Neste tutorial, introduzimos `astropy.coordinates` como uma forma de armazenar e representar as coordenadas astronômicas do céu. Usamos objetos de coordenadas, por meio da interface de classe `SkyCoord`, para analisar e alterar as representações e unidades de coordenadas. Também demonstramos como usar um objeto `SkyCoord` com o pacote `astroquery` para consultar um banco de dados astronômico, o arquivo científico *Gaia*. Em seguida, criamos um único objeto `SkyCoord` com os dados consultados que representam as coordenadas do céu de muitos objetos. Finalmente, introduzimos o conceito de usar `astropy.coordinates` para representar uma posição 3D de um objeto ou conjunto de objetos.